In [2]:
import fit_classes as fp
import numpy as np
import os
from scipy.interpolate import interp1d
from scipy import optimize

errors = []
ERROR = 0.1 # Error in mV

class DataPlus(fp.Data):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        def sinus_model(params, x):
            A,f,phi,c = params
            return A*np.sin(2*np.pi*f*x + phi) + c
        
        file = os.path.join(os.path.abspath(os.getcwd()), "data", "params.txt")
        name = str(self.name).split("\\")[-1]
        var = None
        vals = []
        with open(file, "r") as file:
            for line in file:
                vals.append((
                    line.strip().split(": ")[0],
                     tuple(map(float, line.strip().split(": ")[-1].strip("()").split(", ")))
                    ))

        check = False
        for var in vals:
            if var[0] == name:
                A, f, phi, c = var[1]
                check = True
                
        if check == False:
            A, f, phi, c = 100, 0.006, -13.06097, 25
            A = (np.abs(max(self.y)) + np.abs(min(self.y)))/2
            c = (max(self.y) + min(self.y))/2

            f_interp = interp1d(self.x, self.y, kind='cubic')
            x_dense = np.linspace(self.x.min(), self.x.max(), 1000)
            y_dense = f_interp(x_dense)

            crossings = []
            for i in range(len(x_dense)-1):
                if (y_dense[i] - c) * (y_dense[i+1] - c) < 0:  # Sign change
                    # Refine crossing with brentq
                    root = optimize.brentq(lambda x_val: f_interp(x_val) - c, x_dense[i], x_dense[i+1])
                    crossings.append(root)
                    # punt
                    # if self.x[list(self.x).index(punt) - 5] > self.x[list(self.x).index(punt) + 5]:
                    #     dalende_root.append(punt)

            
            # Find descending zero-crossings
            stijgende_root = []
            for i in range(len(self.y) - 2):
                if (self.y[i - 1] + self.y[i - 2])/2 < c and (self.y[i + 1] + self.y[i + 2])/2 >= c and self.x[i] > 10:
                    if stijgende_root != []:
                        if self.x[i] - stijgende_root[-1] > 10:
                            stijgende_root.append(self.x[i])
                    else:
                        stijgende_root.append(self.x[i])
            
            crossing_count = len(crossings)
            # crossing_count = crossing_count if crossing_count < 30 else 30
            # f = 0.00101048 if abs(len(crossings) - 21) < abs(len(crossings) - 42) else 0.00239959
            f = 0.003
            # phi  = -(stijgende_root[0]*f)*2*np.pi
            phi = 0
            # A = 8

        # phi = np.arcsin(self.x[0] / A)

        FIT_POINTS = 400
        SKIP_POINTS = 1

        self.x = self.x[:FIT_POINTS][::SKIP_POINTS]
        self.y = self.y[:FIT_POINTS][::SKIP_POINTS]
        self.dx = self.dx[:FIT_POINTS][::SKIP_POINTS]
        self.dy = self.dy[:FIT_POINTS][::SKIP_POINTS]

        self.fit = self.fit(sinus_model, initial_guess=(A, f, phi, c))
        self.fit.init = A, f, phi, c

        if check == False:
            file = os.path.join(os.path.abspath(os.getcwd()), "data", "params.txt")
            if self.fit.chi2_red < 10:
                values = (float(A), float(f), float(phi), float(c))
                with open(file, "a") as file:
                    file.write(f"{name}: {values}\n")


        self.A, self.f, self.phi, self.c = self.fit.minima

def data_from_file(file):
    file_path = os.path.join(os.path.dirname(os.path.abspath(os.getcwd())), file)
    data = np.genfromtxt(file_path, delimiter=",", skip_header=9)
    x = data[:, 0].astype(int)
    y = data[:, 1]
    mask = np.concatenate(([True], y[1:] != y[:-1]))
    x = x[mask]
    y = y[mask]

    dy = ERROR*np.ones_like(y)
    return DataPlus(x, y, dy, name=file_path)
        
def get_lowest_folders(root):
    lowest_folders = []
    for dirpath, dirnames, filenames in os.walk(root):
        if not dirnames:
            lowest_folders.append(dirpath)
    return lowest_folders

def load_data_by_folder(root):
    folder_data = {}
    total_files = 0
    for folder in get_lowest_folders(root):
        total_files += len([f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))])

    current_file = 0
    for folder in get_lowest_folders(root):
        folder_name = os.path.basename(folder)
        data_list = []
        for file in os.listdir(folder):
            file_path = os.path.join(folder, file)
            if os.path.isfile(file_path):
                current_file += 1
                # os.system(f'echo "File: {current_file:>3}/{total_files}"')
                data_list.append(data_from_file(file_path))
        
        if data_list:
            folder_data[folder_name] = np.array(data_list)
    return folder_data

data_arrays = load_data_by_folder(os.path.join(os.path.abspath(os.getcwd()), "data"))


# # print(data_arrays['opgave1'][0].show())
# print(f"{data_arrays['opgave1'][0].fit}")
# print(f"{data_arrays['opgave1'][0].f}")
# # print(f"{data_arrays['opgave1'][0].A}")
# # print(f"{data_arrays['opgave1'][0].w}")
# data_arrays['opgave1'][0].fit.show(size=2, fit_guess=True)
# print(f"{data_arrays['opgave5_freq2'][13].fit}")
# print(f"{data_arrays['opgave5_freq2'][13].f}")
# print(str(data_arrays['2000Hz_26_1cm'][27].name).split('\\')[-1])
# data_arrays['2000Hz_26_1cm'][27].fit.show(size=2, fit_guess=True)
# print((data_arrays['2000Hz_44_1cm'][10].name).split('\\')[-1])
# data_arrays['2000Hz_44_1cm'][10].fit.show(size=2, fit_guess=True)
# print((data_arrays['2000Hz_44_1cm'][4].name).split('\\')[-1])
# data_arrays['2000Hz_44_1cm'][4].fit.show(size=2, fit_guess=True)

